<a href="https://colab.research.google.com/github/babelfishz/colab-models/blob/master/orchid_gd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

In [0]:
!pip install tf-nightly

In [0]:
!pip install --upgrade tensorflow
#!pip install tf-nightly

In [0]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import  Dropout, Input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
train_path = './gdrive/My Drive/dataSet/train'
valid_path = './gdrive/My Drive/dataSet/val'
test_path = './gdrive/My Drive/dataSet/test'

train_batches = ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),  batch_size=32)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path,target_size=(224,224),  batch_size=32)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224),  batch_size=32)

Found 188 images belonging to 3 classes.
Found 18 images belonging to 3 classes.
Found 18 images belonging to 3 classes.


In [0]:
vgg16_model = tensorflow.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))
for layer in vgg16_model.layers[:-4]:
    layer.trainable = False
    
# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(vgg16_model)
 
# Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=tensorflow.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = model.fit(train_batches, 
            epochs=5, 
            validation_data=valid_batches, 
            steps_per_epoch=train_batches.samples/train_batches.batch_size,
            validation_steps=valid_batches.samples/valid_batches.batch_size,
            verbose=1)

In [0]:
test_imgs, test_labels = next(test_batches)

predictions = model.predict(test_imgs)

print(predictions)

In [0]:
tf.keras.models.save_model(model, "./gdrive/My Drive/_models/flower/1")
print("models written")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./gdrive/My Drive/_models/flower/1/assets
models written


In [0]:
saved_model = tf.keras.models.load_model("./gdrive/My Drive/_models/flower/1")
input_shape = (None, 224, 224, 3)
saved_model.build(input_shape)
saved_model.summary()

#imported = tf.saved_model.load("/content/gdrive/My Drive/Models/flower/1")



In [0]:
test_imgs, test_labels = next(test_batches)

predictions = saved_model.predict(test_imgs)

print(predictions)

In [0]:
def preprocess_b64_image(image_bytes, h=224, w=224):
    image = tf.io.decode_base64(image_bytes[0])
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, size=[h, w])
    #image = (image - 127.5) / 127.5
    image = tf.expand_dims(image, axis=0)
    return image

#image_bytes = Input(shape = (1,),dtype="string")
image_bytes = tf.keras.Input(shape=[], batch_size=1, name='b64_image_bytes', dtype=tf.string)
preprocessed_image = preprocess_b64_image(image_bytes)

In [0]:
predictions = saved_model(preprocessed_image)
new_model = tf.keras.Model(image_bytes, predictions)
new_model.summary()
print('Model Input Shape:', new_model.input_shape)

In [0]:

def preprocess_and_decode(img_str, new_shape=[224,224]):
    img = tf.io.decode_base64(img_str)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, new_shape, method=tf.image.ResizeMethod.BILINEAR)
    # if you need to squeeze your input range to [0,1] or [-1,1] do it here
    return img
InputLayer = Input(shape = (1,),dtype="string")
OutputLayer = tf.keras.layers.Lambda(lambda img : tf.map_fn(lambda im : preprocess_and_decode(im[0]), img, dtype="float32"))(InputLayer)
base64_model = tf.keras.Model(InputLayer,OutputLayer)


In [0]:
base64_input = base64_model.input
final_output = saved_model(base64_model.output)
new_model = tf.keras.Model(base64_input,final_output)

new_model.summary()

In [0]:
tf.keras.models.save_model(new_model, "./gdrive/My Drive/_models/flower/2")

INFO:tensorflow:Assets written to: ./gdrive/My Drive/_models/flower/2/assets


In [0]:
new_saved_model = tf.keras.models.load_model("./gdrive/My Drive/_models/flower/2")
new_saved_model.summary()

In [26]:

import base64
import json
import tensorflow as tf

url = "http://47.88.158.181:8501/v1/models/flower:predict"

with open('3.jpg', "rb") as img_file:
  encoded_string = base64.urlsafe_b64encode(img_file.read()).decode("utf-8")
data = json.dumps({"instances": [{"b64_image_bytes": encoded_string}]})

r = requests.post(url = url, data = data)
r.json()


{'predictions': [[0.000492823776, 3.64402258e-05, 0.99947077]]}

In [0]:
# plots images with labels within jupyter notebook
def plots(ims, figsize=(24,12), rows=6, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=32)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

#imgs, labels = next(train_batches)
#plots(imgs, titles=labels)

In [14]:
!saved_model_cli show --dir ./gdrive/My\ Drive/_models/flower/2 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['b64_image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (1)
        name: serving_default_b64_image_bytes:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['sequential'] tensor_info:
        dtype: DT_FLOAT
        shape: (1, 3)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0413 13:57:52.241466 140193101354880 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/